# Graph Neural Network (GNN) for Solvent Prediction

**Hypothesis**: GNNs capture molecular structure that tabular features miss. The GNN benchmark achieved CV 0.0039 (5x better than our best CV 0.008194). GNNs might have a DIFFERENT CV-LB relationship.

**Implementation**:
- Use PyTorch Geometric with `from_smiles` utility
- Represent molecules as graphs (atoms as nodes, bonds as edges)
- Use GCNConv for message passing
- Combine graph embeddings with reaction conditions (temperature, time)
- Use global_mean_pool for graph-level readout

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch Geometric imports
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_smiles
from torch_geometric.data import Data, Batch

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)  # Use float32 for GNN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load SMILES lookup
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {SMILES_DF.shape}')
print(SMILES_DF.head())

SMILES lookup: (26, 1)
                                           solvent smiles
SOLVENT NAME                                             
Cyclohexane                                      C1CCCCC1
Ethyl Acetate                                   O=C(OCC)C
Acetic Acid                                       CC(=O)O
2-Methyltetrahydrofuran [2-MeTHF]              O1C(C)CCC1
1,1,1,3,3,3-Hexafluoropropan-2-ol  C(C(F)(F)F)(C(F)(F)F)O


In [4]:
# Pre-compute graph data for all solvents
SOLVENT_GRAPHS = {}
for solvent in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent, 'solvent smiles']
    # Handle mixture solvents (e.g., "Water.Acetonitrile")
    if '.' in smiles:
        # For mixtures, we'll handle them separately
        parts = smiles.split('.')
        graphs = [from_smiles(s) for s in parts]
        SOLVENT_GRAPHS[solvent] = graphs
    else:
        SOLVENT_GRAPHS[solvent] = from_smiles(smiles)

print(f'Pre-computed graphs for {len(SOLVENT_GRAPHS)} solvents')

# Check a few examples
for solvent in ['Methanol', 'Ethanol', 'Water.Acetonitrile']:
    if solvent in SOLVENT_GRAPHS:
        g = SOLVENT_GRAPHS[solvent]
        if isinstance(g, list):
            print(f'{solvent}: mixture of {len(g)} components')
        else:
            print(f'{solvent}: x={g.x.shape}, edge_index={g.edge_index.shape}')

Pre-computed graphs for 26 solvents
Methanol: x=torch.Size([2, 9]), edge_index=torch.Size([2, 2])
Ethanol: x=torch.Size([3, 9]), edge_index=torch.Size([2, 4])
Water.Acetonitrile: mixture of 2 components


In [5]:
# GNN Model - Hybrid approach with tabular features
class GNNEncoder(nn.Module):
    """Encodes a molecular graph into a fixed-size embedding."""
    def __init__(self, node_features=9, hidden_dim=64, output_dim=32):
        super().__init__()
        self.conv1 = GCNConv(node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        
    def forward(self, x, edge_index, batch):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)  # Graph-level representation
        return x


class HybridGNNModel(nn.Module):
    """Hybrid model combining GNN embeddings with tabular features."""
    def __init__(self, node_features=9, hidden_dim=64, graph_dim=32, tabular_dim=145, output_dim=3):
        super().__init__()
        self.gnn = GNNEncoder(node_features, hidden_dim, graph_dim)
        # Combine graph embedding with tabular features
        combined_dim = graph_dim + tabular_dim
        self.fc1 = nn.Linear(combined_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        
    def forward(self, graph_data, tabular_features):
        # Get graph embedding
        graph_emb = self.gnn(graph_data.x, graph_data.edge_index, graph_data.batch)
        # Concatenate with tabular features
        x = torch.cat([graph_emb, tabular_features], dim=1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x


print('Hybrid GNN model defined')

GNN model defined


In [6]:
# Load tabular feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

GNN model wrapper defined


In [7]:
# Hybrid GNN Model Wrapper with train_model and predict methods
class HybridGNNModelWrapper:
    def __init__(self, data='single', epochs=300, lr=0.001, hidden_dim=128, graph_dim=64):
        self.data = data
        self.epochs = epochs
        self.lr = lr
        self.hidden_dim = hidden_dim
        self.graph_dim = graph_dim
        self.model = None
        self.scaler = StandardScaler()
        self.tabular_dim = 2 + 3 + SPANGE_DF.shape[1] + DRFP_FILTERED.shape[1] + ACS_PCA_DF.shape[1]  # 145
        
    def _get_tabular_features(self, X, flip=False):
        """Extract tabular features (Spange + DRFP + ACS PCA + kinetics)."""
        # Kinetics features
        temp_c = X["Temperature"].values.reshape(-1, 1)
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([temp_c, time_m, inv_temp, log_time, interaction])
        
        if self.data == 'single':
            X_spange = SPANGE_DF.loc[X["SOLVENT NAME"]].values
            X_drfp = DRFP_FILTERED.loc[X["SOLVENT NAME"]].values
            X_acs = ACS_PCA_DF.loc[X["SOLVENT NAME"]].values
        else:
            # Mixed solvent - weighted combination
            A_spange = SPANGE_DF.loc[X["SOLVENT A NAME"]].values
            B_spange = SPANGE_DF.loc[X["SOLVENT B NAME"]].values
            A_drfp = DRFP_FILTERED.loc[X["SOLVENT A NAME"]].values
            B_drfp = DRFP_FILTERED.loc[X["SOLVENT B NAME"]].values
            A_acs = ACS_PCA_DF.loc[X["SOLVENT A NAME"]].values
            B_acs = ACS_PCA_DF.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def _get_graph_batch(self, X, flip=False):
        """Create a batch of graphs for the given samples."""
        graphs = []
        for idx, row in X.iterrows():
            if self.data == 'single':
                solvent = row["SOLVENT NAME"]
                g = SOLVENT_GRAPHS.get(solvent)
                if g is None or isinstance(g, list):
                    g = from_smiles('C')
            else:
                solvent_a = row["SOLVENT A NAME"]
                solvent_b = row["SOLVENT B NAME"]
                
                g_a = SOLVENT_GRAPHS.get(solvent_a)
                g_b = SOLVENT_GRAPHS.get(solvent_b)
                
                if g_a is None or isinstance(g_a, list):
                    g_a = from_smiles('C')
                if g_b is None or isinstance(g_b, list):
                    g_b = from_smiles('C')
                
                if flip:
                    g = g_b
                else:
                    g = g_a
            
            g_copy = Data(
                x=g.x.float(),
                edge_index=g.edge_index,
                edge_attr=g.edge_attr if hasattr(g, 'edge_attr') else None
            )
            graphs.append(g_copy)
        
        return Batch.from_data_list(graphs)
    
    def train_model(self, X, Y):
        """Train the hybrid GNN model."""
        # Get tabular features
        tabular = self._get_tabular_features(X)
        tabular_scaled = self.scaler.fit_transform(tabular)
        
        # Get graph batch
        graph_batch = self._get_graph_batch(X)
        
        # Convert to tensors
        tabular_tensor = torch.tensor(tabular_scaled, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(Y.values, dtype=torch.float32).to(device)
        graph_batch = graph_batch.to(device)
        
        # Initialize model
        self.model = HybridGNNModel(
            node_features=9,
            hidden_dim=self.hidden_dim,
            graph_dim=self.graph_dim,
            tabular_dim=self.tabular_dim,
            output_dim=3
        ).to(device)
        
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=30, factor=0.5)
        criterion = nn.MSELoss()
        
        # Training loop
        self.model.train()
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            
            predictions = self.model(graph_batch, tabular_tensor)
            loss = criterion(predictions, y_tensor)
            
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
            
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 50:
                    break
        
        return self
    
    def predict(self, X):
        """Make predictions."""
        self.model.eval()
        
        # Get tabular features
        tabular = self._get_tabular_features(X)
        tabular_scaled = self.scaler.transform(tabular)
        
        # Get graph batch
        graph_batch = self._get_graph_batch(X)
        
        # Convert to tensors
        tabular_tensor = torch.tensor(tabular_scaled, dtype=torch.float32).to(device)
        graph_batch = graph_batch.to(device)
        
        with torch.no_grad():
            predictions = self.model(graph_batch, tabular_tensor)
        
        # TTA for mixed solvents
        if self.data == 'full':
            tabular_flip = self._get_tabular_features(X, flip=True)
            tabular_flip_scaled = self.scaler.transform(tabular_flip)
            tabular_flip_tensor = torch.tensor(tabular_flip_scaled, dtype=torch.float32).to(device)
            graph_batch_flip = self._get_graph_batch(X, flip=True).to(device)
            with torch.no_grad():
                predictions_flip = self.model(graph_batch_flip, tabular_flip_tensor)
            predictions = (predictions + predictions_flip) / 2
        
        predictions = torch.clamp(predictions, 0, 1)
        
        return predictions


print('Hybrid GNN model wrapper defined')
print(f'Tabular feature dimension: {2 + 3 + SPANGE_DF.shape[1] + DRFP_FILTERED.shape[1] + ACS_PCA_DF.shape[1]}')

Single solvent data: X=(656, 3), Y=(656, 3)


Test predictions shape: torch.Size([50, 3])
Test predictions range: [0.1061, 0.8808]


In [8]:
# Quick test of hybrid model
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on a small subset
X_test = X_single.iloc[:50]
Y_test = Y_single.iloc[:50]

model = HybridGNNModelWrapper(data='single', epochs=50)
model.train_model(X_test, Y_test)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')
print(f'Test predictions range: [{preds.min():.4f}, {preds.max():.4f}]')


=== Single Solvent CV ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:00<00:14,  1.59it/s]

  8%|▊         | 2/24 [00:01<00:13,  1.62it/s]

 12%|█▎        | 3/24 [00:01<00:12,  1.62it/s]

...

 92%|█████████▏| 22/24 [00:13<00:01,  1.64it/s]

 96%|█████████▌| 23/24 [00:14<00:00,  1.56it/s]

100%|██████████| 24/24 [00:14<00:00,  1.56it/s]

100%|██████████| 24/24 [00:14<00:00,  1.61it/s]

Single Solvent MSE: 0.021116 (n=656)


In [9]:
# Run CV on single solvent data with hybrid model
print('\n=== Single Solvent CV (Hybrid GNN) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = HybridGNNModelWrapper(data='single', epochs=300, lr=0.001)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.cpu().numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')


=== Full Data CV ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:00<00:08,  1.43it/s]

 15%|█▌        | 2/13 [00:01<00:07,  1.39it/s]

 23%|██▎       | 3/13 [00:02<00:07,  1.41it/s]

...

 85%|████████▍ | 11/13 [00:07<00:01,  1.47it/s]

 92%|█████████▏| 12/13 [00:08<00:00,  1.47it/s]

100%|██████████| 13/13 [00:09<00:00,  1.41it/s]

100%|██████████| 13/13 [00:09<00:00,  1.44it/s]

Full Data MSE: 0.018319 (n=1227)


In [10]:
# Run CV on full data with hybrid model
print('\n=== Full Data CV (Hybrid GNN) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = HybridGNNModelWrapper(data='full', epochs=300, lr=0.001)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.cpu().numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')


=== CV SCORE SUMMARY ===
Single Solvent MSE: 0.021116 (n=656)
Full Data MSE: 0.018319 (n=1227)
Overall MSE: 0.019293

Best CV (exp_032): 0.008194
GNN Benchmark: 0.0039

✗ WORSE: 135.46% worse than best CV


In [ ]:
# Update submission cells to use HybridGNNModelWrapper
# First update cell 11 (third last cell)
pass  # This cell will be replaced

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GNNModelWrapper(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Final CV score verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')
print(f'GNN Benchmark: 0.0039')